# Cuaderno 16: Modelo compacto MTZ para el ATSP

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido **completo** $D=(V,A)$, con $V=\{1, \ldots, n\}$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$.

El *problema del agente viajero asimétrico (Asymmetric Traveling Salesman Problem, ATSP)* consiste en encontrar un circuito dirigido que visite **todos** los nodos de $D$ y que tenga el menor costo posible.

Una manera alternativa de evitar subciclos en la solución, que no requiere un número exponencial de restricciones, fue propuesta por [C.E.Miller, A.W.Tucker y R.A.Zemlin en 1960](https://dl.acm.org/doi/abs/10.1145/321043.321046?casa_token=wS9ir40FaVQAAAAA%3AREqLlEQWddvpSw0sEq2toPwlctyB4Tfa2O2aUHj3WqIfPlXESLWYFByDmy1UWEqcdIFZG6RibRhLXw). La formulación compacta *Miller-Tucker-Zemlin (MTZ)* consiste en introducir variables auxiliares de ordenamiento $u_i$ para los nodos $i \in V \setminus \{ 1 \}$, que indican la posición de cada nodo dentro del tour, asumiendo que el nodo 1 ocupa la primera posición. Utilizando además las variables binarias $x_{ij}$ de selección para indicar los arcos que forman parte del tour, el problema del agente viajero puede formularse como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(j, i) \in A} x_{ji} = 1, \quad \forall i \in V,\\
&\sum_{(i, j) \in A} x_{ij} = 1, \quad \forall i \in V,\\ 
& u_j \geq u_i + (1 + n) x_{ij} - n, \quad\forall (i,j) \in A, i \neq 1, j \neq 1, \\
& x_{ij} \in \{0, 1\}, \quad \forall (i, j) \in A,\\
& u_i \in \{2, \ldots, n \} \quad \forall i \in V \setminus \{1\}.\\
\end{align*}

La función objetivo mide el costo total de los arcos seleccionados en el tour.

Las dos primeras familias de restricciones corresponden a las restricciones de grado de los nodos.

La tercera familia de restricciones requiere que, para cualquier arco $(i,j)$ cuyos dos extremos sean distintos al nodo $1$, si el arco es seleccionado dentro de la solución, entonces debe cumplirse que $u_j \geq u_i + 1$. Notar que de esta manera se evitan soluciones que incluyan ciclos que no contienen al nodo 1. Como resultado, se eliminan soluciones con subtoures, empleando únicamente $m -2n +2$ restricciones, donde $m=n(n-1)$ es el número de arcos en el grafo.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los datos. Usaremos la función `randint` para generar puntos con coordenadas aleatorias en el rango {0,..,100}. El costo de un arco $(i,j)$ será igual a la distancia euclideana entre $i$ y $j$:

In [ ]:
from gurobipy import *
import random as rm
import math 

# Numero de nodos del grafo
n = 200

# Nodos del grafo
V = tuplelist(range(1,n+1))

# Nodos sin el primero
V2 = V[1:]

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()


Emplearemos el módulo `matplotlib` para graficar el tour de la solución. Definiremos para ello la función `dibujarTour` que recibe tres argumentos: una lista `coordx` con las coordenadas horizontales de los nodos, una lista `coordy` con las coordenadas verticales y un vector `tour` con una permutación de los nodos indicando el orden de visita en la solución.

In [ ]:
import matplotlib.pyplot as plt
import random

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    display(plt.show())


Definimos ahora el objeto modelo, las variables y la función objetivo. Notar que las variables de ordenamiento son enteros en el rango $\{2,\ldots, n\}$ y que existe una variable para cada nodo excepto el nodo 1.

In [ ]:
# Crear el objeto modelo
m = Model('atsp-compacto')

# Crear las variables de seleccion de arcos
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# Crear las variables de ordenamiento de nodos
u = m.addVars(V2, name="u", vtype=GRB.INTEGER, lb=2, ub=n)

# Crear la funcion objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)


Añadimos las restricciones de grado:

In [ ]:
# Restricciones de grado saliente
m.addConstrs((x.sum(i,'*')  == 1 for i in V), 
                 "g_saliente")
    
# Restricciones de grado entrante
m.addConstrs((x.sum('*', i)  == 1 for i in V), 
                 "g_entrante")

Para eliminar los subciclos, empleamos las restricciones que fijan valores a las variables de ordenamiento

In [ ]:
# Restricciones de ordenamiento de nodos
m.addConstrs((u[j]  >= u[i] + (1 + n)*x[i,j] - n 
              for i,j in A if i!=1 and j!=1), "ordenamiento");

Escribimos el modelo a un archivo de texto:

In [ ]:
# Escribir el modelo a un archivo
m.write('atsp-mtz.lp')


Establecer un tiempo límite para la solución del modelo y una tolerancia para la brecha de optimalidad:

In [ ]:
# Terminar al alcanzar un Gap del 1%
m.Params.MIPGap = 0.05

# Terminar luego de 180 segundos
m.Params.TimeLimit = 180

Finalmente, resolvemos el modelo y mostramos la solución. Notar que antes de mostrar la solución, verificamos el valor de la propiedad `SolCount` del objeto del modelo, para determinar si en la ejecución de `optimize` fue posible encontrar al menos una solución factible.

In [ ]:
# Calcular la solucion optima
m.optimize()

# Escribir la solucion
if m.SolCount > 0:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nTour optimo:')
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{} -> {}'.format(i, j))


In [ ]:
# Crear lista con arcos seleccionados en la solucion
vx = m.getAttr('x', x)
L = [(i,j) for i,j in A if vx[i,j]>=0.95]
print(L)

# Recuperar el tour como un ordenamiento de los nodos
T = [1]
# nodo actual:
i = 1
while True:
    # Determinar sucesor de i
    j = [j for j in V if (i,j) in L].pop()
    # Colocar sucesor en la lista del tour y actualizar i
    T.append(j)
    i = j
    # Terminar cuando el nodo colocado sea 1
    if i==1: break;
    
print("Tour óptimo: {}".format(T))
        
# Graficar el tour
dibujarTour(coordx, coordy, T)    

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Modelo compacto para el problema del agente viajero asimétrico (ATSP)

# Luis M. Torres (EPN 2022)

from gurobipy import *
import random as rm
import matplotlib.pyplot as plt

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)

# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de nodos del grafo
n = 200

# Nodos del grafo
V = tuplelist(range(1,n+1))

# Nodos sin el primero
V2 = V[1:]

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()

try:
    # Crear el objeto modelo
    m = Model('tsp')

    # Crear las variables de seleccion de arcos
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Crear las variables de ordenamiento de nodos
    u = m.addVars(V2, name="u", vtype=GRB.INTEGER, lb=2, ub=n)

    # Crear la funcion objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    # Restricciones de grado saliente
    m.addConstrs((x.sum(i,'*')  == 1 for i in V), 
                 "g_saliente")
    
    # Restricciones de grado entrante
    m.addConstrs((x.sum('*', i)  == 1 for i in V), 
                 "g_entrante")

    # Restricciones de ordenamiento de nodos
    m.addConstrs((u[j]  >= u[i] + (1 + n)*x[i,j] - n 
                  for i,j in A if i!=1 and j!=1), "ordenamiento")

    # Escribir el modelo a un archivo
    # m.write('tsp.lp')

    # Terminar al alcanzar un Gap del 5%
    m.Params.MIPGap = 0.05

    # Terminar luego de 180 segundos
    m.Params.TimeLimit = 180    # Calcular la solucion optima
    
    # Desactivar todos los cortes
    m.Params.Cuts = 3
    
    m.optimize()

    # Escribir la solucion
    # Proceder solamente si se tiene al menos una solución factible
    if m.SolCount > 0:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nTour optimo:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} -> {}'.format(i, j))
                
        # Crear lista con arcos seleccionados en la solucion
        L = [(i,j) for i,j in A if vx[i,j]>=0.99]

        # Recuperar el tour como un ordenamiento de los nodos
        T = [1]
        # nodo actual:
        i = 1
        while True:
            # Determinar sucesor de i
            j = [j for j in V if (i,j) in L].pop()
            # Colocar sucesor en la lista del tour y actualizar i
            T.append(j)
            i = j
            # Terminar cuando el nodo colocado sea 1
            if i==1: break;
        
        # Graficar el tour
        dibujarTour(coordx, coordy, T)    
    
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

In [ ]:
display(dibujarTour(coordx, coordy, T) )

In [ ]:
vx = m.getAttr('x', x)
# Crear lista con arcos seleccionados en la solucion
L = [(i,j) for i,j in A if vx[i,j]>=0.99]
print(L)

# Recuperar el tour como un ordenamiento de los nodos
T = [1]
# nodo actual:
i = 1
while True:
    # Determinar sucesor de i
    j = [j for j in V if (i,j) in L].pop()
    # Colocar sucesor en la lista del tour y actualizar i
    T.append(j)
    i = j
    # Terminar cuando el nodo colocado sea 1
    if i==1: break;
        
# Graficar el tour
dibujarTour(coordx, coordy, T)    
